In [8]:
import requests
import sqlite3
import time
from bs4 import BeautifulSoup


start_time = time.time()

db = "quotes_data.db"
conn = sqlite3.connect(db)
cursor = conn.cursor()

# Step 2: Create or Reset Table
cursor.execute("DROP TABLE IF EXISTS scraped_quotes")
cursor.execute("""
    CREATE TABLE IF NOT EXISTS scraped_quotes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        quote TEXT,
        author TEXT,
        tags TEXT
    )
""")
conn.commit()

print("-------------------------------------------------------------")
print(f"Connected to database: {db}")


URL = "http://quotes.toscrape.com"
try:
    print(f"1. Fetching data from: {URL} ...")
    response = requests.get(URL, headers={"User-Agent": "Mozilla/5.0"})


    if response.status_code == 200:
        print("2. Page loaded successfully!")
        

        soup = BeautifulSoup(response.text, "html.parser")

        quotes = soup.find_all("div", class_="quote")
        print(f"3. Found {len(quotes)} quotes!")
        

        saved_count = 0
        for quote in quotes:
            quote_text = quote.find("span", class_="text").get_text(strip=True)
            author = quote.find("small", class_="author").get_text(strip=True)
            tags = ", ".join(tag.get_text() for tag in quote.find_all("a", class_="tag"))

            cursor.execute("INSERT INTO scraped_quotes (quote, author, tags) VALUES (?, ?, ?)",
                           (quote_text, author, tags))
            saved_count += 1
        
        conn.commit()
        print(f"4. {saved_count} quotes saved to the database!\n")
    
    else:
        print(f"Failed to load the page. Status code: {response.status_code}")

except Exception as e:
    print(f"Error: {e}")

finally:
    cursor.close()
    conn.close()
    print("Database connection closed.")
print("-------------------------------------------------------------")


print("--------------- Previewing saved quotes: ----------------")
conn = sqlite3.connect(db)
cursor = conn.cursor()

cursor.execute("SELECT * FROM scraped_quotes LIMIT 10")
for row in cursor.fetchall():
    print(f"\"{row[1]}\" - {row[2]} [Tags: {row[3]}]\n")

cursor.close()
conn.close()

end_time = time.time()
execution_time = end_time - start_time
print(f"--------------- Execution Time: {execution_time:.2f} seconds -----------------")


-------------------------------------------------------------
Connected to database: quotes_data.db
1. Fetching data from: http://quotes.toscrape.com ...
2. Page loaded successfully!
3. Found 10 quotes!
4. 10 quotes saved to the database!

Database connection closed.
-------------------------------------------------------------
--------------- Previewing saved quotes: ----------------
"“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”" - Albert Einstein [Tags: change, deep-thoughts, thinking, world]

"“It is our choices, Harry, that show what we truly are, far more than our abilities.”" - J.K. Rowling [Tags: abilities, choices]

"“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”" - Albert Einstein [Tags: inspirational, life, live, miracle, miracles]

"“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intole